En este vídeo analizaremos las interfaces de programas de aplicaciones (API, para abreviar). Específicamente, discutiremos qué es una API, bibliotecas API, y API REST, incluidas Solicitud y Respuesta y un ejemplo con PyCoinGecko.

### ¿Qué es una API?

Una API permite que dos piezas de software hablen entre sí. Por ejemplo, tienes tu programa, tienes algunos datos, y tienes otros componentes de software. Utiliza la API para comunicarte con otro software a través de entradas y salidas. Al igual que una función, no es necesario saber cómo funciona la API, solo sus entradas y salidas.

### Ejemplo con Pandas

Pandas es en realidad un conjunto de componentes de software, muchos de los cuales ni siquiera están escritos en Python. Supongamos que tienes algunos datos y un conjunto de componentes de software. Utilizamos la API de pandas para procesar los datos comunicándonos con los otros componentes del software.

**Código de ejemplo:**

In [2]:

import pandas as pd

# Crear un diccionario
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6]
}

# Crear un DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Calcular la media de las columnas
print(df.mean())


   A  B
0  1  4
1  2  5
2  3  6
A    2.0
B    5.0
dtype: float64


## API REST

Las API REST son otro tipo popular de API; te permiten comunicarte a través de Internet, lo que te permite aprovechar recursos como almacenamiento, acceder a más datos, algoritmos de inteligencia artificial y mucho más. El RE significa Representacional, la S para Estado y la T para Transferencia.

En las API REST, tu programa se llama cliente. La API se comunica con un servicio web al que llamas a través de Internet. Existen reglas para la comunicación, entrada o solicitud, y salida o respuesta.

Términos comunes:

- Cliente: Tu código o programa.
- Recurso: El servicio web al que te conectas.
- Punto final: La URL del servicio.
- Solicitud y Respuesta: La comunicación entre cliente y servicio.

## Métodos HTTP

Los métodos HTTP son una forma de transmitir datos a través de Internet. Les decimos a las API REST qué hacer enviando una solicitud, que normalmente se comunica a través de un mensaje HTTP. Este mensaje suele contener un archivo JSON con instrucciones sobre qué operación realizar.

Código de ejemplo usando `requests`:

In [1]:
import requests

# Enviar una solicitud GET a una API
response = requests.get('https://api.coingecko.com/api/v3/simple/price', params={'ids': 'bitcoin', 'vs_currencies': 'usd'})

# Obtener la respuesta en formato JSON
data = response.json()

print(data)


{'bitcoin': {'usd': 56676}}


#### **Ejemplo con PyCoinGecko**
Usaremos el cliente/envoltorio PyCoinGecko para la API CoinGecko, actualizado cada minuto por CoinGecko.

Código de ejemplo:

In [11]:
!pip install pycoingecko

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import time
from datetime import datetime, timedelta

# Crear un objeto de cliente
cg = CoinGeckoAPI()

# esta linea muestra los metodos y atribuos del objeto creado cg, opcional y 
# como refresco de la lección objetos
'''metodos = (dir(cg))
for metodo in metodos:
    print(metodo)'''
    
# Calcular el tiempo de inicio y fin
end_time = int(time.time())  # Tiempo actual en segundos desde la época
start_time = int((datetime.now() - timedelta(days=30)).timestamp())  # Tiempo 30 días atrás


In [20]:
# Obtener datos históricos de Bitcoin de los últimos 30 días
#data = cg.get_coin_market_chart_range_range_by_id_range_range('bitcoin', 'usd', from_timestamp=30*24*60*60)
bitcoin_data = cg.get_coin_market_chart_range_by_id(
    'bitcoin', 
    'usd', 
    from_timestamp=start_time, 
    to_timestamp=end_time
    )

# Extraer el precio de los datos
prices = bitcoin_data['prices']

# Convertir a DataFrame
df = pd.DataFrame(prices, columns=['timestamp', 'price'])

# Convertir la marca de tiempo a formato legible
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())


       timestamp         price                    date
0  1723395619476  60181.831588 2024-08-11 17:00:19.476
1  1723399850643  60224.756927 2024-08-11 18:10:50.643
2  1723404070667  60080.736166 2024-08-11 19:21:10.667
3  1723407524657  58934.909267 2024-08-11 20:18:44.657
4  1723410805910  58371.554543 2024-08-11 21:13:25.910


**Crear un gráfico de velas**
Para obtener los datos de las velas diarias, agruparemos por fecha para encontrar el mínimo, máximo, primer y último precio de cada día. Usaremos `plotly` para crear el gráfico de velas.

Código de ejemplo:

In [23]:
# Instalar la biblioteca plotly
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import plotly.graph_objects as go

# Agrupar los datos por fecha y calcular OHLC
df.set_index('date', inplace=True)
ohlc = df.resample('D').agg({'price': ['first', 'last', 'min', 'max']})

# Crear gráfico de velas
fig = go.Figure(data=[go.Candlestick(x=ohlc.index,
                                     open=ohlc['price']['first'],
                                     high=ohlc['price']['max'],
                                     low=ohlc['price']['min'],
                                     close=ohlc['price']['last'])])

# Mostrar gráfico
fig.show()


Ahora puedes ver el gráfico de velas abriendo el archivo HTML generado y haciendo clic en "Confíe en HTML" en la parte superior izquierda de la pestaña. ¡Debería verse así!